## Commercial refrigerator
Problem 6-107, Thermodynamics: An Engineering Approach, Yunus Cengel, Michael Boles, 9th Edtition

A commercial refrigerator with refrigerant-134a as the working fluid is used to keep the refrigerated space at -35°C by rejecting waste heat to cooling water that enters the condenser at 18°C at a rate of 0.25 kg/s and leaves at 26°C. The refrigerant enters the condenser at 1.2 MPa and 50°C and leaves at the same pressure subcooled by 5°C. If the compressor consumes 3.3 kW of power, determine (a) the mass flow rate of the refrigerant, (b) the refrigeration load, (c) the COP, and (d) the minimum power input to the compressor for the same refrigeration load.

In [1]:
!pip install CoolProp

In [2]:
from CoolProp.CoolProp import PropsSI

In [3]:
!pip install iapws

  Using cached iapws-1.5.4-py3-none-any.whl


In [4]:
from iapws import IAPWS97

In [5]:
fluid = 'R134a'

In [6]:
# state 2: After the compressor
P2_Pa = 1.2e6 # Pa
T2_K = 50+273.15
h2_JperKg = PropsSI('H','T',T2_K,'P',P2_Pa,fluid); print(h2_JperKg)

426413.7574665965


In [8]:
# state 3: After the condenser
P3_Pa = 1.2e6 # Pa
x3SatLiq = 0
T3sat_K = PropsSI('T','P',P3_Pa,'Q',x3SatLiq,fluid)
subcooling_K = 5

T3_K = T3sat_K - subcooling_K
h3_JperKg = PropsSI('H','T',T3_K,'P',P3_Pa,fluid); print(h3_JperKg)

# h4_JperKg = PropsSI('H','P',P4_Pa,'Q',x4,fluid); print(h4_JperKg)

258345.19360806086


$\dot{Q}_\text{H} = \dot{m}_\text{R134a} \left(h_2 - h_3 \right)$

So we need to determine the heat transferred to the cooling water. For that we can determine the properties of the water entering and exiting the condenser. 

In [15]:
# mass flow rate of water
mdotWater_kgPerS = 0.25

# state 5: Water entering the condenser
T5_K = 18+273.15
P5_MPa = 0.1 # Atmospheric pressure

water5 = IAPWS97(P=P5_MPa, T=T5_K)
h5_kJperKg = water5.h; print(h5_kJperKg)

# state 6: Water exiting the condenser
T6_K = 26 + 273.15
P6_MPa = 0.1 # Atmospheric pressure

water6 = IAPWS97(P=P6_MPa, T=T6_K)
h6_kJperKg = water6.h; print(h6_kJperKg)

75.64073127346839
109.10974020285225


$\dot{Q}_\text{H} = \dot{m}_\text{water} \left(h_6 - h_5 \right)$

In [16]:
QdotH_kW = mdotWater_kgPerS*(h6_kJperKg - h5_kJperKg); print(QdotH_kW)
QdotH_W = QdotH_kW*1000

8.367252232345965


Now that we know the heat transferred out from the hot side of the refrigerator, and the refrigerant properties before and after the condenser, we can determine the mass flow rate of the refrigerant:

In [19]:
mdotR134a_kgPerS = QdotH_W/(h2_JperKg - h3_JperKg)
print('(a) mdot-R134a = {0:1.6f} kg/s.'.format(mdotR134a_kgPerS))

(a) mdot-R134a = 0.049785 kg/s.


For the refrigeration load, we can use this formula:
$W_\text{net,in} = Q_\text{H} - Q_\text{L}$

as we are given the work input and we have determined the heat rejected to the hot side of the refrigerator.

In [23]:
WdotIn_W = 3300
QdotL_W = QdotH_W - WdotIn_W
print('(b) QdotL = {0:1.1f} W.'.format(QdotL_W))

(b) QdotL = 5067.3 W.


Recall that:
$\text{COP} = \frac{\dot{Q}_\text{L}}{\dot{W}_\text{net,in}}$

In [25]:
COP = QdotL_W / WdotIn_W
print('(c) COP = {0:1.2f}.'.format(COP))

(c) COP = 1.54.


The refrigeration space is kept at $T_\text{L} = $-35 $^\circ$C. We know the value for $T_\text{H}$. Therefore, we can determine the COP for the Carnot cycle. 

$\text{COP}_\text{R,rev} = \frac{1}{T_\text{H}/T_\text{L} - 1}$

By knowning the value of $Q_\text{L}$ from above, we can determine the minimum power input to the system. 

In [26]:
TL_K = -35 + 273.15
TH_K = T2_K

In [27]:
COP_Rrev = 1/((TH_K/TL_K)-1); print(COP_Rrev)

2.8017647058823534


In [28]:
WdotInMinimum_W = QdotL_W / COP_Rrev
print('(d) Wdot_minimum = {0:1.2f}.'.format(WdotInMinimum_W))

(d) Wdot_minimum = 1808.59.
